In [1]:
NAME = "PA"
FIPS = 42

In [2]:
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, ForeignKey
from sqlalchemy.dialects.mysql import LONGTEXT, VARCHAR, TINYBLOB
import math

In [3]:
engine = create_engine('mysql+pymysql://Pistons:ASIO0qw1!dsij01/a?@mysql3.cs.stonybrook.edu:3306/Pistons', echo=True, pool_pre_ping=True, pool_recycle=3600, pool_size=20)
meta = MetaData()

In [4]:
# Blocks = Table(
#    'Blocks', meta, 
#     Column('ID', String(255), primary_key = True), 
#     Column('COUNTY', String(255)), 
#     Column('NUMBER', String(255)), 
#     Column('CD', String(255)), 
# )
# Districts = Table(
#    'Districts', meta, 
#     Column('id', String(255), primary_key = True),
#     Column('cd', String(255)), 
#     Column('geometry', LONGTEXT), 
#     Column('districtingId', String(255),ForeignKey('Districtings.id')), 
#     Column('electionId', String(255), ForeignKey('Elections.id')), 
#     Column('populationId', String(255), ForeignKey('Populations.id')), 
#     Column('vapId', String(255), ForeignKey('VotingAgePopulations.id')),
# )
# States = Table(
#    'States', meta, 
#     Column('id', String(255), primary_key = True), 
#     Column('geometry', LONGTEXT), 
#     Column('name', String(255)), 
# )
# Districtings = Table(
#    'Districtings', meta, 
#     Column('id', String(255), primary_key = True), 
#     Column('electionId', String(255), ForeignKey('Elections.id')), 
#     Column('populationId', String(255), ForeignKey('Populations.id')), 
#     Column('stateId', String(255), ForeignKey('States.id')), 
#     Column('vapId', String(255), ForeignKey('VotingAgePopulations.id')),
# )
CensusBlocks = Table(
   'CensusBlocks', meta, 
    Column('id', String(255), primary_key = True),
    Column('geometry', LONGTEXT), 
    Column('electionId', String(255), ForeignKey('Elections.id')), 
    Column('populationId', String(255), ForeignKey('Populations.id')), 
    Column('precinctId', String(255), ForeignKey('Precincts.id')), 
    Column('vapId', String(255), ForeignKey('VotingAgePopulations.id')),
)
Precincts = Table(
   'Precincts', meta, 
    Column('id', String(255), primary_key = True),
    Column('districtId', String(255)), 
    Column('geometry', LONGTEXT), 
    Column('electionId', String(255), ForeignKey('Elections.id')), 
    Column('populationId', String(255), ForeignKey('Populations.id')), 
    Column('vapId', String(255), ForeignKey('VotingAgePopulations.id')),
    Column('county', String(255)),
)
Districts = Table(
   'Districts', meta, 
    Column('id', String(255), primary_key = True),
    Column('cd', String(255)), 
    Column('geometry', LONGTEXT), 
    Column('districtingId', String(255), ForeignKey('Districtings.id')), 
    Column('electionId', String(255), ForeignKey('Elections.id')), 
    Column('populationId', String(255), ForeignKey('Populations.id')), 
    Column('vapId', String(255), ForeignKey('VotingAgePopulations.id')),
)
County = Table(
    'County', meta, 
    Column('id', String(255), primary_key = True),
    Column('state', String(255)), 
    Column('county', String(255)), 
    Column('geometry', LONGTEXT), 
)
# DistrictsGeo = Table(
#    'DistrictsGeo', meta, 
#     Column('id', String(255), primary_key = True),
#     Column('geometry', LONGTEXT), 
# )
Districtings = Table(
   'Districtings', meta, 
    Column('id', String(255), primary_key = True),
    Column('electionId', String(255), ForeignKey('Elections.id')), 
    Column('populationId', String(255), ForeignKey('Populations.id')), 
    Column('stateId', String(255), ForeignKey('States.id')),
    Column('vapId', String(255), ForeignKey('VotingAgePopulations.id')),
)
States = Table(
   'States', meta, 
    Column('id', String(255), primary_key = True),
    Column('geometry', LONGTEXT), 
    Column('name', String(255)), 
     Column('enactedId', String(255), ForeignKey('Districtings.id')),
)
Elections = Table(
   'Elections', meta, 
    Column('id', String(255), primary_key = True), 
    Column('democraticVotes', Integer), 
    Column('republicanVotes', Integer), 
    Column('otherVotes', Integer), 
    Column('totalVotes', Integer), 
    Column('type', String(255)), 
)
Populations = Table(
    'Populations', meta, 
    Column('id', String(255), primary_key = True), 
    Column('TOTAL', Integer), 
    Column('HISPA', Integer), 
    Column('WHITE', Integer), 
    Column('BLACK', Integer), 
    Column('NATIV', Integer), 
    Column('ASIAN', Integer),  
    Column('OTHER', Integer), 
)
VotingAgePopulations = Table(
    'VotingAgePopulations', meta, 
    Column('id', String(255), primary_key = True), 
    Column('TOTAL', Integer), 
    Column('HISPA', Integer), 
    Column('WHITE', Integer), 
    Column('BLACK', Integer), 
    Column('NATIV', Integer), 
    Column('ASIAN', Integer),  
    Column('OTHER', Integer), 
)
meta.create_all(engine)

2021-12-13 23:39:39,644 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-12-13 23:39:39,645 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-12-13 23:39:39,661 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-12-13 23:39:39,661 INFO sqlalchemy.engine.Engine [generated in 0.00065s] {}
2021-12-13 23:39:39,695 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2021-12-13 23:39:39,695 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-12-13 23:39:39,744 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-12-13 23:39:39,746 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2021-12-13 23:39:39,746 INFO sqlalchemy.engine.Engine [generated in 0.00050s] {'table_schema': 'pistons', 'table_name': 'CensusBlocks'}
2021-12-13 23:39:39,762 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_na

# Reading

In [5]:
block = gpd.read_file('./{name}_Block.geojson'.format(name=NAME))
district = gpd.read_file('./{name}_District.geojson'.format(name=NAME))
precint = gpd.read_file('./{name}_Precints.geojson'.format(name=NAME))
state = gpd.read_file('./{name}_State.geojson'.format(name=NAME))
districting = pd.read_csv('./{name}_Districting.csv'.format(name=NAME))

In [ ]:
county = gpd.read_file('./{name}_County.geojson'.format(name=NAME))

In [ ]:
precint = gpd.read_file('./{name}_Precints.geojson'.format(name=NAME))

In [ ]:
precint

In [ ]:
districting.rename(columns = {"Unnamed: 0":"index"}, inplace = True)

In [ ]:
levels = [block,district,precint,districting]

# Population

In [ ]:
def getPopulation(levels):
    temp = levels[0].loc[:, "TOTAL":"OTHER"].join(levels[0]["index"])
    for i in range(1,len(levels)):
        temp = pd.concat([temp,levels[i].loc[:, "TOTAL":"OTHER"].join(levels[i]["index"])])
    return temp

In [ ]:
population = getPopulation(levels)

In [ ]:
population

In [ ]:
population.rename(columns = {"index":"id"}, inplace = True)

In [ ]:
conn = engine.connect()
conn.execute(Populations.insert(),population.to_dict('records'))

# Election

In [ ]:
# [block,district,precint,districting]

In [ ]:
districting

In [ ]:
def getElection(levels):
    temp = levels[0].loc[:, "democraticVotes":"type"].join(levels[0]["index"])
    for i in range(1,len(levels)):
        temp = pd.concat([temp,levels[i].loc[:, "democraticVotes":"type"].join(levels[i]["index"])])
    return temp

In [ ]:
election = getElection(levels)

In [ ]:
election

In [ ]:
election.rename(columns = {"index":"id"}, inplace = True)

In [ ]:
conn = engine.connect()
conn.execute(Elections.insert(),election.to_dict('records'))

# VAP

In [ ]:
# [block,district,precint,districting]

In [ ]:
def getVAP(levels):
    temp = levels[0].loc[:, "VAP_TOTAL":"VAP_OTHER"].join(levels[0]["index"])
    for i in range(1,len(levels)):
        temp = pd.concat([temp,levels[i].loc[:, "VAP_TOTAL":"VAP_OTHER"].join(levels[i]["index"])])
    return temp

In [ ]:
vap = getVAP(levels)

In [ ]:
vap.rename(columns = {"index":"id",
                      "VAP_TOTAL":"TOTAL",
                      "VAP_HISPA":"HISPA",
                      "VAP_WHITE":"WHITE",
                      "VAP_BLACK":"BLACK",
                      "VAP_NATIV":"NATIV",
                      "VAP_ASIAN":"ASIAN",
                      "VAP_OTHER":"OTHER",
                     }, inplace = True)

In [ ]:
conn = engine.connect()
conn.execute(VotingAgePopulations.insert(),vap.to_dict('records'))

# State

In [ ]:
# States = Table(
#    'K', meta, 
#     Column('id', String(255), primary_key = True), 
#     Column('geometry', LONGTEXT), 
#     Column('name', String(255)), 
# )
# meta.create_all(engine)

In [ ]:
state.drop(["index"], axis = 1,inplace = True)
state = state.to_wkt()

In [ ]:
state

In [ ]:
conn = engine.connect()
conn.execute(States.insert(),state.to_dict('records'))

# Districting

In [ ]:
# Districtings = Table(
#    'K', meta, 
#     Column('id', String(255), primary_key = True), 
#     Column('electionId', String(255), ForeignKey('Elections.id')), 
#     Column('populationId', String(255), ForeignKey('Populations.id')), 
#     Column('stateId', String(255), ForeignKey('States.id')), 
#     Column('vapId', String(255), ForeignKey('VotingAgePopulations.id')),
# )
# meta.create_all(engine)

In [ ]:
districting

In [ ]:
dist = districting[['index']]
dist["electionId"] = dist['index']
dist["populationId"] = dist['index']
dist["stateId"] = FIPS
dist["vapId"] = dist['index']

In [ ]:
dist.rename(columns = {"index":"id"}, inplace = True)

In [ ]:
dist

In [ ]:
list(dist)

In [ ]:
dist.vapId.dtype

In [ ]:
conn = engine.connect()
conn.execute(Districtings.insert(),dist.to_dict('records'))

# County

In [ ]:
county

In [ ]:
county.explore()

In [ ]:
county = county.to_wkt()

In [ ]:
county.rename(columns = {"STATE":"state",
                         "COUNTY":"county"}, inplace = True)

In [ ]:
county.to_file('./{f}county.geojson'.format(f=NAME), driver='GeoJSON') 

In [ ]:
conn = engine.connect()
conn.execute(County.insert(),county.to_dict('records'))

# Districts

In [ ]:
# Districts = Table(
#    'Districts', meta, 
#     Column('id', String(255), primary_key = True),
#     Column('cd', String(255)), 
#     Column('geometry', LONGTEXT), 
#     Column('districtingId', String(255),ForeignKey('Districtings.id')), 
#     Column('electionId', String(255), ForeignKey('Elections.id')), 
#     Column('populationId', String(255), ForeignKey('Populations.id')), 
#     Column('vapId', String(255), ForeignKey('VotingAgePopulations.id')),
# )
# meta.create_all(engine)

In [ ]:
district

In [ ]:
dis = district[['index','CD','geometry']]

In [ ]:
dis['districtingId'] = str(FIPS)+"PL0"
dis['electionId'] = dis['index']
dis['populationId'] = dis['index']
dis['vapId'] = dis['index']

In [ ]:
# dis.CD.astype(str)
dis.rename(columns = {"index":"id",
                      "CD":"cd"}, inplace = True)

In [ ]:
dis = dis.to_wkt()

In [ ]:
dis.geometry.dtype

In [ ]:
dis.to_file('./{f}district.geojson'.format(f=NAME), driver='GeoJSON') 

In [ ]:
conn = engine.connect()
conn.execute(Districts.insert(),dis.to_dict('records'))

# Precint

In [ ]:
# Precincts = Table(
#    'Precincts', meta, 
#     Column('id', String(255), primary_key = True),
#     Column('districtId', String(255)), 
#     Column('geometry', LONGTEXT), 
#     Column('electionId', String(255), ForeignKey('Elections.id')), 
#     Column('populationId', String(255), ForeignKey('Populations.id')), 
#     Column('vapId', String(255), ForeignKey('VotingAgePopulations.id')),
# )

In [ ]:
precint

In [ ]:
prec = precint[['index','CD','geometry']]
prec['districtId'] = str(FIPS) + "PL0D" + prec.CD.astype(str)
prec['electionId'] = prec['index']
prec['populationId'] = prec['index']
prec['vapId'] = prec['index']
prec['county'] = precint['COUNTY']

In [ ]:
prec.drop(["CD"], axis = 1,inplace = True)

In [ ]:
prec.rename(columns = {"index":"id"}, inplace = True)

In [ ]:
prec = prec.to_wkt()

In [ ]:
prec

In [ ]:
prec.geometry.dtype

In [ ]:
prec.to_file('./{f}precinct.geojson'.format(f=NAME), driver='GeoJSON') 

In [ ]:
conn = engine.connect()
conn.execute(Precincts.insert(),prec.to_dict('records'))

# Blocks

In [ ]:
# CensusBlocks = Table(
#    'CensusBlocks', meta, 
#     Column('id', String(255), primary_key = True),
#     Column('geometry', LONGTEXT), 
#     Column('electionId', String(255), ForeignKey('Elections.id')), 
#     Column('populationId', String(255), ForeignKey('Populations.id')), 
#     Column('precinctId', String(255), ForeignKey('Precincts.id')), 
#     Column('vapId', String(255), ForeignKey('VotingAgePopulations.id')),
# )
# meta.create_all(engine)

In [ ]:
block

In [ ]:
b = block[['index','geometry','PrecintID']]
b['electionId'] = b['index']
b['populationId'] = b['index']
b['vapId'] = b['index']

In [ ]:
b.rename(columns = {"index":"id",
                    "PrecintID":"precinctId",
                    }, inplace = True)
b = b.to_wkt()

In [ ]:
b

In [ ]:
conn = engine.connect()
conn.execute(CensusBlocks.insert(),b.to_dict('records'))

# CBNeigh

In [ ]:
CensusBlockNeighbors = Table(
   'CensusBlockNeighbors', meta, 
    Column('blockId', String(255), ForeignKey('CensusBlocks.id'), primary_key = True),
    Column('neighborId', String(255), ForeignKey('CensusBlocks.id'), primary_key = True)
)
# PrecinctNeighbors = Table(
#    'PrecinctNeighbors', meta, 
#     Column('precinctId', String(255), primary_key = True),
#     Column('neighborId', String(255), ForeignKey('Precincts.id'), primary_key = True)
# )
meta.create_all(engine)

In [ ]:
block

In [ ]:
test = block[['NEIGHBORS']]
ind = block[['index']]

In [ ]:
cbn = test.apply(lambda x: pd.Series(x['NEIGHBORS'].split(",")), axis=1).stack().reset_index(level=1, drop=True)
cbn = cbn.to_frame(name="NEIGHBORS")
# cbn["index"] = str(FIPS) + "B" + cbn.index.astype(str)
cbn = cbn[cbn['NEIGHBORS'] != '']

In [ ]:
cbn = pd.merge(cbn, ind, left_index=True, right_index=True)

In [ ]:
cbn.rename(columns = {"index":"blockId",
                      "NEIGHBORS":"neighborId",
                      }, inplace = True)

In [ ]:
cbn

In [ ]:
conn = engine.connect()
conn.execute(CensusBlockNeighbors.insert(),cbn.to_dict('records'))

In [ ]:
# print(i)

In [ ]:
# for x in range(chunks):
#     df_sliced = cbn.iloc[i:j]
#     conn = engine.connect()
#     conn.execute(CensusBlockNeighbors.insert(),df_sliced.to_dict('records'))
# #     print(df_sliced.to_dict('records'))
#     # df_list.append(df_sliced)
# #     df_sliced.to_excel('worldcities_CHUNK-BY-ROWS_'+str(i)+'.xlsx', index=None)
    
#     i += expected_rows
#     j += expected_rows

# PrecinctNeighbors

In [ ]:
PrecinctNeighbors = Table(
   'PrecinctNeighbors', meta, 
    Column('precinctId', String(255), primary_key = True),
    Column('neighborId', String(255), ForeignKey('Precincts.id'), primary_key = True)
)
meta.create_all(engine)

In [ ]:
precint

In [ ]:
test = precint[['NEIGHBORS']]

In [ ]:
test

In [ ]:
pn = test.apply(lambda x: pd.Series(x['NEIGHBORS'].split(",")), axis=1).stack().reset_index(level=1, drop=True)
pn = pn.to_frame(name="NEIGHBORS")
pn["index"] = str(FIPS) + "P" + pn.index.astype(str)

In [ ]:
pn

In [ ]:
pn.rename(columns = {"index":"precinctId",
                      "NEIGHBORS":"neighborId",
                      }, inplace = True)

In [ ]:
conn = engine.connect()
conn.execute(PrecinctNeighbors.insert(),pn.to_dict('records'))

# PrecinctCensusBlocks

In [6]:
PrecinctCensusBlocks = Table(
   'PrecinctCensusBlocks', meta, 
    Column('precinctId', String(255), primary_key = True),
    Column('censusBlockId', String(255), ForeignKey('CensusBlocks.id'), primary_key = True)
)
meta.create_all(engine)

2021-12-13 23:40:56,383 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-12-13 23:40:56,385 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2021-12-13 23:40:56,385 INFO sqlalchemy.engine.Engine [cached since 76.64s ago] {'table_schema': 'pistons', 'table_name': 'CensusBlocks'}
2021-12-13 23:40:56,402 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2021-12-13 23:40:56,403 INFO sqlalchemy.engine.Engine [cached since 76.66s ago] {'table_schema': 'pistons', 'table_name': 'Precincts'}
2021-12-13 23:40:56,421 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2021-12-13 23:40:56,421 INFO sqlalchemy.engine.Engine [cached since 76.68s ago] {'table_schema': 'pistons', 'table_name': 'Districts'}
2021-12-

In [7]:
block

,index,COUNTY,NUMBER,CD,TOTAL,HISPA,WHITE,BLACK,NATIV,ASIAN,...,VAP_OTHER,NEIGHBORS,BORDER,PrecintID,democraticVotes,republicanVotes,otherVotes,totalVotes,type,geometry
0,42B0,CLINTON,100,12,6,1,4,0,0,1,...,0,"42B103350,42B276872,42B77265",0,42P3034,2.981132,8.094340,0.075472,11.150943,PRE2020,"POLYGON ((-77.37753 41.18033, -77.37658 41.180..."
1,42B1,BEAVER,985,17,35,0,28,1,0,0,...,5,"42B36915,42B299843,42B47463,42B326339,42B321179",0,42P1590,15.206897,22.068966,0.620690,37.896552,PRE2020,"POLYGON ((-80.26329 40.60001, -80.26281 40.600..."
2,42B2,BEAVER,435,17,75,4,67,0,0,0,...,3,"42B36985,42B310517,42B297382,42B110387,42B1052...",0,42P1535,12.633333,19.466667,0.200000,32.300000,PRE2020,"POLYGON ((-80.34373 40.72152, -80.34371 40.721..."
3,42B3,BEAVER,975,17,27,4,20,3,0,0,...,0,"42B220718,42B239414,42B23660",0,42P1589,13.677419,16.064516,0.322581,30.064516,PRE2020,"POLYGON ((-80.27489 40.59180, -80.27482 40.592..."
4,42B4,BEAVER,1460,17,0,0,0,0,0,0,...,0,"42B108026,42B165853,42B176364,42B86883",0,42P1503,5.350000,3.000000,0.150000,8.500000,PRE2020,"POLYGON ((-80.29442 40.70585, -80.29429 40.705..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336980,42B336980,BRADFORD,610,12,136,0,127,2,0,1,...,6,"42B101577,42B125382,42B236012,42B293675,42B322795",0,42P2002,2.224719,6.314607,0.146067,8.685393,PRE2020,"POLYGON ((-76.37585 41.77522, -76.37576 41.775..."
336981,42B336981,BRADFORD,530,12,5,0,4,0,0,1,...,0,"42B117100,42B215405,42B252304,42B312090,42B321...",0,42P1993,2.452830,9.150943,0.226415,11.830189,PRE2020,"POLYGON ((-76.53683 41.81024, -76.53651 41.810..."
336982,42B336982,BRADFORD,180,12,19,0,19,0,0,0,...,0,"42B44313,42B117625,42B184750,42B222907,42B332604",0,42P1954,1.375000,5.982143,0.071429,7.428571,PRE2020,"POLYGON ((-76.27715 41.76221, -76.27670 41.762..."
336983,42B336983,BRADFORD,280,12,0,0,0,0,0,0,...,0,"42B2670,42B28923,42B146498,42B207726,42B251832",0,42P1965,0.888889,3.569444,0.152778,4.611111,PRE2020,"POLYGON ((-76.14922 41.86879, -76.14877 41.868..."


In [8]:
pcb = block[['index',"PrecintID"]]

In [9]:
pcb.rename(columns = {"PrecintID":"precinctId",
                      "index":"censusBlockId",
                      }, inplace = True)

/home/diwenliu/anaconda3/envs/geo-env/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [10]:
pcb

,censusBlockId,precinctId
0,42B0,42P3034
1,42B1,42P1590
2,42B2,42P1535
3,42B3,42P1589
4,42B4,42P1503
...,...,...
336980,42B336980,42P2002
336981,42B336981,42P1993
336982,42B336982,42P1954
336983,42B336983,42P1965


In [11]:
# expected_rows = 100000
# chunks = math.floor(len(pcb)/expected_rows + 1)

# # Slice the dataframe...
# i = 0
# j = expected_rows
# for x in range(chunks):
#     df_sliced = pcb.iloc[i:j]
    
#     # df_list.append(df_sliced)
# #     df_sliced.to_excel('worldcities_CHUNK-BY-ROWS_'+str(i)+'.xlsx', index=None)
#     conn = engine.connect()
#     conn.execute(PrecinctCensusBlocks.insert(),df_sliced.to_dict('records'))
#     i += expected_rows
#     j += expected_rows

In [12]:
pcb

,censusBlockId,precinctId
0,42B0,42P3034
1,42B1,42P1590
2,42B2,42P1535
3,42B3,42P1589
4,42B4,42P1503
...,...,...
336980,42B336980,42P2002
336981,42B336981,42P1993
336982,42B336982,42P1954
336983,42B336983,42P1965


In [13]:
pcb.loc[pcb['precinctId'] == "24P0"]

,censusBlockId,precinctId


In [14]:
conn = engine.connect()
conn.execute(PrecinctCensusBlocks.insert(),pcb.to_dict('records'))

2021-12-13 23:40:59,008 INFO sqlalchemy.engine.Engine INSERT INTO `PrecinctCensusBlocks` (`precinctId`, `censusBlockId`) VALUES (%(precinctId)s, %(censusBlockId)s)
2021-12-13 23:40:59,017 INFO sqlalchemy.engine.Engine [generated in 1.21552s] ({'precinctId': '42P3034', 'censusBlockId': '42B0'}, {'precinctId': '42P1590', 'censusBlockId': '42B1'}, {'precinctId': '42P1535', 'censusBlockId': '42B2'}, {'precinctId': '42P1589', 'censusBlockId': '42B3'}, {'precinctId': '42P1503', 'censusBlockId': '42B4'}, {'precinctId': '42P1466', 'censusBlockId': '42B5'}, {'precinctId': '42P1559', 'censusBlockId': '42B6'}, {'precinctId': '42P6341', 'censusBlockId': '42B7'}  ... displaying 10 of 336985 total bound parameter sets ...  {'precinctId': '42P1965', 'censusBlockId': '42B336983'}, {'precinctId': '42P1995', 'censusBlockId': '42B336984'})
2021-12-13 23:43:01,807 INFO sqlalchemy.engine.Engine COMMIT


# PrecinctBorderBlocks

In [15]:
PrecinctBorderBlocks = Table(
   'PrecinctBorderBlocks', meta, 
    Column('precinctId', String(255), primary_key = True),
    Column('borderBlockId', String(255), ForeignKey('CensusBlocks.id'), primary_key = True)
)
meta.create_all(engine)

2021-12-13 23:43:02,009 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-12-13 23:43:02,010 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2021-12-13 23:43:02,011 INFO sqlalchemy.engine.Engine [cached since 202.3s ago] {'table_schema': 'pistons', 'table_name': 'CensusBlocks'}
2021-12-13 23:43:02,029 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2021-12-13 23:43:02,030 INFO sqlalchemy.engine.Engine [cached since 202.3s ago] {'table_schema': 'pistons', 'table_name': 'Precincts'}
2021-12-13 23:43:02,049 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2021-12-13 23:43:02,049 INFO sqlalchemy.engine.Engine [cached since 202.3s ago] {'table_schema': 'pistons', 'table_name': 'Districts'}
2021-12-

In [16]:
test = precint[['BorderBlock']]

In [17]:
precint

,index,COUNTY,NUMBER,CD,TOTAL,HISPA,WHITE,BLACK,NATIV,ASIAN,...,VAP_OTHER,democraticVotes,republicanVotes,otherVotes,totalVotes,type,NEIGHBORS,BORDER,BorderBlock,geometry
0,42P0,ADAMS,000020,13,867,175,670,9,0,5,...,4,175,275,6,456,PRE2020,"42P5,42P3",0,,"POLYGON ((-77.29536 39.91578, -77.29595 39.915..."
1,42P1,ADAMS,000030,13,736,265,428,8,1,0,...,15,102,202,8,312,PRE2020,42P8,0,,"POLYGON ((-77.24497 39.96963, -77.24619 39.969..."
2,42P2,ADAMS,000050,13,1225,231,930,13,3,4,...,28,193,366,7,566,PRE2020,42P3,0,,"POLYGON ((-77.24917 39.92549, -77.25279 39.925..."
3,42P3,ADAMS,000070,13,2550,246,2197,8,3,10,...,54,371,1045,16,1432,PRE2020,"42P0,42P2,42P18,42P13,42P5,42P14,42P8",0,,"POLYGON ((-77.26345 39.87315, -77.26474 39.874..."
4,42P4,ADAMS,000170,13,2358,113,2133,22,3,22,...,40,437,1009,18,1464,PRE2020,"42P20,42P30,42P31,42P4185,42P4175,42P5",0,,"POLYGON ((-77.28833 39.85352, -77.28852 39.853..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9234,42P9234,YORK,950,11,6026,215,5041,421,3,143,...,114,1350,2218,39,3607,PRE2020,"42P9123,42P9124,42P9127,42P9128,42P9172,42P917...",1,"42B81530,42B92017,42B143627,42B191891,42B24458...","POLYGON ((-76.73148 39.80507, -76.73167 39.805..."
9235,42P9235,YORK,960,11,2129,37,1895,78,0,30,...,52,415,802,31,1248,PRE2020,42P9169,0,,"POLYGON ((-76.59537 39.74363, -76.59587 39.743..."
9236,42P9236,YORK,970,10,4552,106,4252,17,6,12,...,118,667,2001,31,2699,PRE2020,"42P9077,42P9139,42P9140,42P9149,42P9191,42P919...",0,,"POLYGON ((-76.92599 40.01436, -76.92626 40.014..."
9237,42P9237,YORK,980,10,2573,30,2474,8,2,6,...,33,299,1128,9,1436,PRE2020,"42P7,42P9,42P9139,42P9154,42P9203,42P9236",1,"42B2426,42B11927,42B38494,42B41058,42B54048,42...","POLYGON ((-76.96568 39.93523, -76.96620 39.935..."


In [18]:
pbb = test.loc[test.BorderBlock != ""]

In [19]:
pbb = pbb.apply(lambda x: pd.Series(x['BorderBlock'].split(",")), axis=1).stack().reset_index(level=1, drop=True)
pbb = pbb.to_frame(name="BorderBlock")
pbb["index"] = str(FIPS) + "P" + pbb.index.astype(str)

In [20]:
pbb.rename(columns = {"index":"precinctId",
                      "BorderBlock":"borderBlockId",
                      }, inplace = True)

In [21]:
pbb

,borderBlockId,precinctId
6,42B8467,42P6
6,42B40318,42P6
6,42B187296,42P6
6,42B268238,42P6
7,42B2944,42P7
...,...,...
9237,42B198700,42P9237
9237,42B251345,42P9237
9237,42B282853,42P9237
9237,42B298859,42P9237


In [22]:
conn = engine.connect()
conn.execute(PrecinctBorderBlocks.insert(),pbb.to_dict('records'))

2021-12-13 23:43:03,043 INFO sqlalchemy.engine.Engine INSERT INTO `PrecinctBorderBlocks` (`precinctId`, `borderBlockId`) VALUES (%(precinctId)s, %(borderBlockId)s)
2021-12-13 23:43:03,044 INFO sqlalchemy.engine.Engine [generated in 0.01178s] ({'precinctId': '42P6', 'borderBlockId': '42B8467'}, {'precinctId': '42P6', 'borderBlockId': '42B40318'}, {'precinctId': '42P6', 'borderBlockId': '42B187296'}, {'precinctId': '42P6', 'borderBlockId': '42B268238'}, {'precinctId': '42P7', 'borderBlockId': '42B2944'}, {'precinctId': '42P7', 'borderBlockId': '42B13713'}, {'precinctId': '42P7', 'borderBlockId': '42B20853'}, {'precinctId': '42P7', 'borderBlockId': '42B20894'}  ... displaying 10 of 7938 total bound parameter sets ...  {'precinctId': '42P9237', 'borderBlockId': '42B298859'}, {'precinctId': '42P9237', 'borderBlockId': '42B300335'})
2021-12-13 23:43:06,763 INFO sqlalchemy.engine.Engine COMMIT


# DistrictBorderPrecincts

In [ ]:
DistrictBorderPrecincts = Table(
   'DistrictBorderPrecincts', meta, 
    Column('districtId', String(255), primary_key = True),
    Column('borderPrecinctId', String(255), ForeignKey('Precincts.id'), primary_key = True)
)
meta.create_all(engine)

In [ ]:
test = district[['BorderPrecinct','index']]
dbp = test.loc[test.BorderPrecinct != ""]
dbp.set_index('index', inplace = True)

In [ ]:
dbp = dbp.apply(lambda x: pd.Series(x['BorderPrecinct'].split(",")), axis=1).stack().reset_index(level=1, drop=True)
dbp = dbp.to_frame(name="BorderPrecinct")
dbp["index"] = dbp.index.astype(str)

In [ ]:
dbp

In [ ]:
dbp.rename(columns = {"index":"districtId",
                      "BorderPrecinct":"borderPrecinctId",
                      }, inplace = True)

In [ ]:
conn = engine.connect()
conn.execute(DistrictBorderPrecincts.insert(),dbp.to_dict('records'))

# DistrictNeighbors

In [ ]:
DistrictNeighbors = Table(
   'DistrictNeighbors', meta, 
    Column('districtId', String(255), primary_key = True),
    Column('neighborId', String(255), ForeignKey('Districts.id'), primary_key = True)
)
meta.create_all(engine)

In [ ]:
district

In [ ]:
test = district[['NEIGHBORS','index']]
dn = test.loc[test.NEIGHBORS != ""]
dn.set_index('index', inplace = True)

In [ ]:
dn = dn.apply(lambda x: pd.Series(x['NEIGHBORS'].split(",")), axis=1).stack().reset_index(level=1, drop=True)
dn = dn.to_frame(name="NEIGHBORS")
dn["index"] = dn.index.astype(str)

In [ ]:
dn

In [ ]:
dn.rename(columns = {"index":"districtId",
                     "NEIGHBORS":"neighborId",
                      }, inplace = True)

In [ ]:
conn = engine.connect()
conn.execute(DistrictNeighbors.insert(),dn.to_dict('records'))